# Neural Network Implementation
We finally approach the neural network implementation of this fucking paper

In [56]:
# Packages
import numpy as np
import pandas as pd
import torch
import scipy.stats as stats
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim, torch.nn.functional as F
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler

# All Required DataSets
IVSurface = np.load('testSurface.pkl', allow_pickle = True)
SP500 = np.load('SP500.pkl', allow_pickle = True)
VIX = np.load('VIX.pkl', allow_pickle = True)

In [110]:
mergedData = pd.merge(pd.merge(IVSurface, SP500, how = 'left', on = 'Date'), VIX, how = 'left', on = 'Date')
mergedData = mergedData.filter(['strike', 'daysExp', 'fittedIV', 'Returns', 'VIX'])
mergedData['IVChange'] = mergedData.fittedIV.diff(periods = 441)
mergedData = mergedData.shift(periods = -441).dropna()
mergedData.to_pickle('finalDataSet.pkl')

## Preparing Test Validation Splits

In [104]:
splitFraction = 0.7 # Here we train using 70% of the data
trainSplit = int(splitFraction * mergedData.shape[0])

trainData = mergedData.iloc[:trainSplit]
xTrain = trainData.filter(['strike', 'daysExp', 'Returns', 'VIX']).values
yTrain = trainData.filter(['IVChange']).values

validationData = mergedData.iloc[trainSplit:]
xTest = validationData.filter(['strike', 'daysExp', 'Returns', 'VIX']).values
yTest = validationData.IVChange.values

In [101]:
sc = StandardScaler()
xTrain = sc.fit_transform(xTrain)
xTest = sc.transform(xTest)
 #Not too sure on this normalisation.

array([ 0.00637234,  0.00629518,  0.0061718 , ...,  0.00170169,
       -0.00096076, -0.00304449])

## First Neural Network (3 Factor)
We first implement the simple three-factor model as a standard feed-forward neural network.

In [105]:
hiddenNodes = [100, 100, 100]
firstPredictor = Sequential()

In [106]:
firstPredictor.add(Dense(hiddenNodes[0], kernel_initializer = 'uniform', activation = 'relu', input_dim = 4))

for i in range(len(hiddenNodes) - 1):
    firstPredictor.add(Dense(hiddenNodes[i+1], kernel_initializer = 'uniform', activation = 'relu'))

firstPredictor.add(Dense(1, kernel_initializer = 'uniform', activation = 'relu'))

firstPredictor.compile(optimizer='sgd',
                       loss='mse',
                       metrics=[tf.keras.metrics.MeanSquaredError()])

firstPredictor.fit(xTrain, yTrain, batch_size = 10, epochs = 10)


Epoch 1/10
587/587 [==============================] - 1s 725us/step - loss: 5.9619e-04 - mean_squared_error: 5.9619e-04
Epoch 2/10
587/587 [==============================] - 0s 704us/step - loss: 5.9362e-04 - mean_squared_error: 5.9362e-04
Epoch 3/10
587/587 [==============================] - 0s 711us/step - loss: 5.9362e-04 - mean_squared_error: 5.9362e-04
Epoch 4/10
587/587 [==============================] - 0s 699us/step - loss: 5.9362e-04 - mean_squared_error: 5.9362e-04
Epoch 5/10
587/587 [==============================] - 0s 704us/step - loss: 5.9362e-04 - mean_squared_error: 5.9362e-04
Epoch 6/10
587/587 [==============================] - 0s 710us/step - loss: 5.9362e-04 - mean_squared_error: 5.9362e-04
Epoch 7/10
587/587 [==============================] - 0s 708us/step - loss: 5.9362e-04 - mean_squared_error: 5.9362e-04
Epoch 8/10
587/587 [==============================] - 0s 721us/step - loss: 5.9362e-04 - mean_squared_error: 5.9362e-04
Epoch 9/10
587/587 [====================